In [1]:
#!pip install selenium webdriver-manager tenacity
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
import base64
import os
import time
import platform
import subprocess

# Datas
hoje = datetime.now()
ontem = hoje - timedelta(days=1)

# Formatar datas
data_hoje_str = hoje.strftime("%d-%m-%Y")
data_ontem_str = ontem.strftime("%Y-%m-%d")

# Criar pasta de destino
pasta_destino = os.path.join("chuvas", f"Mapas precipitacao dia {data_hoje_str}")
os.makedirs(pasta_destino, exist_ok=True)

# Inicializar navegador com WebDriver Manager
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver.maximize_window()
    driver.get("https://tempo.inmet.gov.br/PrecAcumulada")
    wait = WebDriverWait(driver, 20)

    # Esperar os botões carregarem
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ui.compact.buttons button.ui.button")))

    # Obter os botões
    botoes = driver.find_elements(By.CSS_SELECTOR, ".ui.compact.buttons button.ui.button")
    periodos = []
    for botao in botoes:
        texto = botao.text.strip().lower().replace(" dias", "").replace(" dia", "")
        periodos.append((texto, botao))

    # Para cada botão, clicar e salvar imagem
    for periodo, botao in periodos:
        try:
            botao.click()
            print(f"[INFO] Clicando no botão: {periodo} dia(s)")
            time.sleep(3)  # esperar imagem carregar

            img = driver.find_element(By.CSS_SELECTOR, "img.ui.big.rounded.image")
            src_base64 = img.get_attribute("src")

            if src_base64.startswith("data:image"):
                _, encoded = src_base64.split(",", 1)
                img_data = base64.b64decode(encoded)

                nome_arquivo = f"imagem_{data_ontem_str}_{periodo}dias.png"
                caminho_completo = os.path.join(pasta_destino, nome_arquivo)

                with open(caminho_completo, "wb") as f:
                    f.write(img_data)

                print(f"[SUCESSO] Salvo: {caminho_completo}")
            else:
                print(f"[ERRO] Imagem inválida no período {periodo} dias.")

        except Exception as e:
            print(f"[ERRO] Falha ao processar {periodo} dias: {e}")

except Exception as e:
    print("[FATAL] Erro geral:", e)

finally:
    driver.quit()

    # Abrir a pasta no final (Windows, macOS, Linux)
    try:
        path = os.path.realpath(pasta_destino)
        if platform.system() == "Windows":
            os.startfile(path)
        elif platform.system() == "Darwin":
            subprocess.Popen(["open", path])
        else:
            subprocess.Popen(["xdg-open", path])
    except Exception as e:
        print(f"[ERRO] Ao tentar abrir a pasta: {e}")

[INFO] Clicando no botão: 1 dia(s)
[SUCESSO] Salvo: chuvas\Mapas precipitacao dia 29-07-2025\imagem_2025-07-28_1dias.png
[INFO] Clicando no botão: 3 dia(s)
[SUCESSO] Salvo: chuvas\Mapas precipitacao dia 29-07-2025\imagem_2025-07-28_3dias.png
[INFO] Clicando no botão: 5 dia(s)
[SUCESSO] Salvo: chuvas\Mapas precipitacao dia 29-07-2025\imagem_2025-07-28_5dias.png
[INFO] Clicando no botão: 10 dia(s)
[SUCESSO] Salvo: chuvas\Mapas precipitacao dia 29-07-2025\imagem_2025-07-28_10dias.png
[INFO] Clicando no botão: 15 dia(s)
[SUCESSO] Salvo: chuvas\Mapas precipitacao dia 29-07-2025\imagem_2025-07-28_15dias.png
[INFO] Clicando no botão: 30 dia(s)
[SUCESSO] Salvo: chuvas\Mapas precipitacao dia 29-07-2025\imagem_2025-07-28_30dias.png
[INFO] Clicando no botão: 90 dia(s)
[SUCESSO] Salvo: chuvas\Mapas precipitacao dia 29-07-2025\imagem_2025-07-28_90dias.png
